In [1]:
# python 有哪些内置 package
import getpass
import os

os.environ["COHERE_API_KEY"] = getpass.getpass()

from langchain_cohere import ChatCohere

# 还有哪些 model
model = ChatCohere(model='command-r')

 ········


In [2]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hi Bob! It's nice to meet you. How's it going today?", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '4e334a7a-8016-40c9-9691-d405490fab47', 'token_count': {'input_tokens': 71, 'output_tokens': 16}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '4e334a7a-8016-40c9-9691-d405490fab47', 'token_count': {'input_tokens': 71, 'output_tokens': 16}}, id='run-6dce921c-8be2-42ca-90b0-22d08783b502-0')

In [3]:
model.invoke([HumanMessage(content="What's my name")])

AIMessage(content="I'm sorry, but as an AI chatbot, I don't know your name. The name is something very personal, and unique to each individual. It's usually given by the parents or chosen by the individual themselves. \n\nDo you want me to help you choose a name, or perhaps give you some suggestions for a name? I can also help you come up with some nicknames, if that's what you're interested in!", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'b92ee0a7-eb9b-43d5-83e9-92c99ff7395d', 'token_count': {'input_tokens': 70, 'output_tokens': 87}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'b92ee0a7-eb9b-43d5-83e9-92c99ff7395d', 'token_count': {'input_tokens': 70, 'output_tokens': 87}}, id='run-f83cebd6-25c7-44a9-b06f-c6338a8eadb8-0')

In [4]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content="Hi, I'm Bob"),
    AIMessage(content="Hi, I'm Alice"),
    HumanMessage(content="What are our names")
])

AIMessage(content='Your name is Bob and mine is Alice. Nice to virtually meet you, Bob!', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'a4555705-76e6-46ba-a59b-69bce6a5b148', 'token_count': {'input_tokens': 86, 'output_tokens': 17}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'a4555705-76e6-46ba-a59b-69bce6a5b148', 'token_count': {'input_tokens': 86, 'output_tokens': 17}}, id='run-11c35830-a845-4b8a-93c2-eae9154121f3-0')

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [6]:
config = {"configurable": {"session_id": "abc2"}}

In [7]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

"Hi Bob! It's nice to meet you. How's it going today?"

In [8]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Bob! Did you forget already?'

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [10]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)
config = {"configurable": {"session_id": "abc5"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response.content

"Hi Jim! It's nice to meet you. How's it going today?"

In [11]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Jim! Did you forget already?'

In [12]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"}
)

response.content

'¡Hola Bob! Mucho gusto en conocerte. Espero que pases un buen rato charlando conmigo. 😊'

In [13]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc11"}}

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)

response.content

'¡Hola, Todd! ¡Bienvenido a mi conversación! Espero que estés teniendo un buen día. 😊'

In [14]:
from langchain_core.runnables import RunnablePassthrough

def filter_messages(messages, k=10):
    return messages[-k:]

chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

In [15]:
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [17]:
# 超出 10 条 message 会被截断
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name")],
        "language": "English"
    }
)
response.content

"I'm sorry, I can't answer that. You haven't introduced yourself yet! Who do you want to be called?"

In [18]:
# 未超出，不会被截断
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my fav ice cream")],
        "language": "English",
    }
)
response.content

"Sorry, I am unable to access your personal preferences, so I don't know what your favorite ice cream is. However, the most popular flavor in the world seems to be chocolate. But vanilla is also a very common favorite, and for good reason! It's delicious and versatile. It pairs well with so many different desserts and sweets, and it's a classic flavor that's hard to dislike."

In [19]:
# 一问一答之后连冰淇淋也不记得了
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="whats my favorite ice cream?")],
        "language": "English",
    },
    config=config,
)

response.content

"That's a good question! However, as an AI chatbot, I don't have access to personal information like that unless you choose to share it with me. \n\nDo you have a favorite ice cream flavor that you'd like to share? I can make some suggestions for delicious ice cream flavors if you'd like!"

In [23]:
# stream output
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="")

Hi Todd! I hope you're having a great day so far! Here's a joke to make you laugh:

What do you call a factory that makes distinctly average products?

An adequate factory!

Have a wonderful day, Todd!